# Confidence intervals

## Tasks

### Task 1

We called 1000 clients and we want to check the probability that 100 or fewer users will agree to renew our subscription.  
When called, a client renews their subscription with a probability of 0.3.

In [1]:
from scipy.stats import binom


n = 1000  
p = 0.3   
k = 100   


probability = binom.cdf(k, n, p)

print(f'The probability of 100 or fewer renewals is: {probability:.4f}')

The probability of 100 or fewer renewals is: 0.0000


### Task 2

What confidence interval would correspond to a 3-sigma rule?

In [2]:
from scipy.stats import norm


probability = 2 * norm.cdf(3) - 1
print(f'The probability within 3 standard deviations is: {probability:.4f}')

The probability within 3 standard deviations is: 0.9973


### Task 3

We are estimating the average daily temperature in a certain city.  
In our sample, there are 63 observations, with a mean value of 25 and a standard deviation of 7 (known in advance, not calculated from the
sample).  
Calculate the 95% confidence interval.

In [3]:
mean_temperature = 25
std = 7
sample_size = 63
ci = 0.95


norm.interval(ci, loc=mean_temperature, scale=std/sample_size**.5)

(23.27147423942126, 26.72852576057874)

### Task 4

Suppose we have 25 stores.  
The average daily revenue is 170 thousand dollars, and the estimated standard deviation based on this sample is 12 thousand dollars.  
Estimate the 95% confidence interval for the average revenue of the stores.

In [4]:
mean_revenue = 170000
std = 12000
stores_number = 25
ci = 0.95


norm.interval(ci, loc=mean_revenue, scale=std/stores_number**.5)

(165296.08643710386, 174703.91356289614)

### Task 5

Let's practice building more confidence intervals.  
As a sample, we will use synthetic data: generate 1000 numbers from a Poisson distribution with lambda parameter 50.   
Calculate the 95% confidence interval for the mean based on the Central Limit Theorem.   
What is the width of the interval you obtained? 

In [5]:
import numpy as np
from scipy import stats


lambda_param = 50
n_samples = 1000
np.random.seed(42) 
data = np.random.poisson(lam=lambda_param, size=n_samples)

sample_mean = np.mean(data)
sample_std = np.std(data, ddof=1)

confidence_level = 0.95
alpha = 1 - confidence_level

z_critical = stats.norm.ppf(1 - alpha / 2)  # equals to 0.975 percentile
standard_error = sample_std / np.sqrt(n_samples)
margin_of_error = z_critical * standard_error

lower_bound = sample_mean - margin_of_error
upper_bound = sample_mean + margin_of_error
interval_width = upper_bound - lower_bound


print(f'Sample Mean: {sample_mean:.2f}')
print(f'Sample Standard Deviation: {sample_std:.2f}')
print(f'Z-critical value for {confidence_level*100}% confidence: {z_critical:.2f}')
print(f'{confidence_level*100}% Confidence Interval: ({lower_bound:.2f}, {upper_bound:.2f})')
print(f'Width of the Confidence Interval: {interval_width:.2f}')

Sample Mean: 49.80
Sample Standard Deviation: 7.28
Z-critical value for 95.0% confidence: 1.96
95.0% Confidence Interval: (49.35, 50.25)
Width of the Confidence Interval: 0.90


### Task 6

Let's use the same dataset (synthetic from the Poisson distribution).  
Build a 95% confidence interval for the standard deviation using bootstrap.  
It is recommended to do it using the corresponding function from the `scipy.stats` library.  
Attention, the bootstrap function has a method parameter that determines how the confidence interval boundaries will be calculated.  In our case, it should be `percentile`.

In [6]:
from scipy.stats import bootstrap


def std_deviation(x, axis=None):
    return np.std(x, ddof=1, axis=axis)


result = bootstrap((data,),
                   statistic=std_deviation,
                   confidence_level=confidence_level,
                   n_resamples=10000,
                   method='percentile',
                   random_state=np.random.seed(42))


print(f'Standard Deviation {confidence_level*100}% Confidence Interval: ({result.confidence_interval.low:.2f}, {result.confidence_interval.high:.2f})')

Standard Deviation 95.0% Confidence Interval: (6.96, 7.58)


### Task 7

Let's use confidence intervals to make more accurate conclusions about the model's metrics.  
Use `California Housing` dataset to solve this task.  
Let's estimate a 95-percent confidence interval for the MSE of Ridge regression for this sample. Use bootstrap for it.  

In [7]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from tqdm import tqdm


# loading data
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

# converting to numpy arrays for easier indexing
X_np = X.values
y_np = y.values

n_samples = len(X_np)
n_bootstrap = 1000
mse_scores = []


for i in tqdm(range(n_bootstrap)):
    # picking indices for train (with replacement)
    bootstrap_indices = np.random.choice(n_samples, n_samples, replace=True)
    
    # getting remaining indices for test
    oob_indices = np.setdiff1d(np.arange(n_samples), np.unique(bootstrap_indices))
    
    # if there are no train samples, continue to next iteration
    if len(oob_indices) == 0:
        continue
    
    # spliting data 
    X_train = X_np[bootstrap_indices]
    y_train = y_np[bootstrap_indices]
    X_test = X_np[oob_indices]
    y_test = y_np[oob_indices]
    
    # training the Ridge regression model
    model = Ridge()
    model.fit(X_train, y_train)
    
    # making predictions and calculating MSE
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)


# calculating 95% confidence interval
lower_bound = np.percentile(mse_scores, 2.5)
upper_bound = np.percentile(mse_scores, 97.5)

print(f'95% Confidence Interval for MSE: [{lower_bound:.4f}, {upper_bound:.4f}]')
print(f'Mean MSE: {np.mean(mse_scores):.4f}')

100%|██████████| 1000/1000 [00:37<00:00, 26.67it/s]

95% Confidence Interval for MSE: [0.5060, 1.1433]
Mean MSE: 0.6561


### Task 8

Let's evaluate how well confidence intervals technique performs.  
Code a function that generates a random sample from a normal distribution:
- mean value of 3, 
- a standard deviation of 4,
- the sample size will be 1000 objects.  

Build the corresponding 95-percent confidence interval for the mean based on the Central Limit Theorem.  
Repeat the procedure of generating a sample and constructing the confidence interval 1000 times.  
Each time, check if the confidence interval actually covers your true mean value.  

Calculate the proportion of cases where the interval covered the true value. What value is your result closest to?

In [8]:
def generate_sample(mean, std, sample_size):
    return np.random.normal(loc=mean, scale=std, size=sample_size)

In [9]:
def calc_ci(sample, ci=0.95):
    sample_mean = np.mean(sample)
    sample_std = np.std(sample, ddof=1)
    
    se = sample_std / np.sqrt(sample_size)
    t_critical = stats.t.ppf(1- ((1-ci)/2), df=sample_size-1)
    
    lower_bound = sample_mean - t_critical * se
    upper_bound = sample_mean + t_critical * se
    
    return lower_bound, upper_bound

In [10]:
mean = 3
std = 4
sample_size = 1000
n_iterations = 1000

cnt = 0
for _ in tqdm(range(n_iterations)):
    sample = generate_sample(mean, std, sample_size)
    lower, upper = calc_ci(sample, ci=0.95)
    
    if lower <= mean <= upper:
        cnt += 1


coverage_proportion = cnt / n_iterations

print(f'Out of {n_iterations} iterations, {cnt} confidence intervals covered the true mean.')
print(f'Proportion of coverage: {coverage_proportion*100:.2f}%')

100%|██████████| 1000/1000 [00:00<00:00, 2886.75it/s]

Out of 1000 iterations, 948 confidence intervals covered the true mean.
Proportion of coverage: 94.80%
